In [32]:
import os
from datetime import datetime, timedelta
import time

import tweepy
from tweepy import OAuthHandler
import pandas as pd

auth = tweepy.OAuthHandler(os.environ.get('twitter_consumer_api_key'), os.environ.get('twitter_consumer_api_secret'))
auth.set_access_token(os.environ.get('twitter_access_api_key'), os.environ.get('twitter_access_api_secret'))

api = tweepy.API(auth, wait_on_rate_limit=True)

In [2]:
username = 'nba'
count = 500
df_user_tweets = pd.DataFrame()

In [11]:
df_test = []  
for tweet in tweepy.Cursor(api.search_tweets, 'nba', count = 500).items(250):
    df_test.append(tweet)

In [13]:
# result_type = 'popular'
df = pd.DataFrame()
try:
  for tweet in tweepy.Cursor(api.search_tweets, 'nba', count = 5000, result_type = 'mixed').items(5000):
      # print(status)
      df = df.append(
                            {'created_at' : tweet._json['created_at'],
                                        'tweet_id': tweet._json['id_str'],
                                        'user': tweet._json['user']['screen_name'],
                                        'user_id': tweet._json['user']['id'],
                                          'tweet': tweet._json['text'],
                      'likes': tweet._json['favorite_count'],
                          'retweets': tweet._json['retweet_count'],
                        'lang': tweet._json['lang'],
                            'scrape_ts': datetime.now(),
          'profile_img': tweet._json['user']['profile_image_url'],
          'url': f"https://twitter.com/twitter/statuses/{tweet._json['id']}",
                          }, ignore_index=True
      )
      
except BaseException as e:
    print('failed on_status,',str(e))
    time.sleep(3)

In [15]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import numpy as np

In [16]:
analyzer = SentimentIntensityAnalyzer()
df["compound"] = [
    analyzer.polarity_scores(x)["compound"] for x in df["tweet"]
]
df["neg"] = [analyzer.polarity_scores(x)["neg"] for x in df["tweet"]]
df["neu"] = [analyzer.polarity_scores(x)["neu"] for x in df["tweet"]]
df["pos"] = [analyzer.polarity_scores(x)["pos"] for x in df["tweet"]]
df["sentiment"] = np.where(df["compound"] > 0, 1, 0)


In [5]:
df_test[0]._json

{'created_at': 'Wed Jul 13 00:34:28 +0000 2022',
 'id': 1547016575903039489,
 'id_str': '1547016575903039489',
 'text': 'RT @6toHombreLATAM: Nunca me voy a cansar de ver esta asistencia de LeBron. Es mi favorita. https://t.co/eXE0wZMUIJ',
 'truncated': False,
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [{'screen_name': '6toHombreLATAM',
    'name': 'Sexto Hombre',
    'id': 1187340909765115904,
    'id_str': '1187340909765115904',
    'indices': [3, 18]}],
  'urls': [],
  'media': [{'id': 1027933534483300352,
    'id_str': '1027933534483300352',
    'indices': [92, 115],
    'media_url': 'http://pbs.twimg.com/amplify_video_thumb/1027933534483300352/img/sibzfqC2WG1EEY3Y.jpg',
    'media_url_https': 'https://pbs.twimg.com/amplify_video_thumb/1027933534483300352/img/sibzfqC2WG1EEY3Y.jpg',
    'url': 'https://t.co/eXE0wZMUIJ',
    'display_url': 'pic.twitter.com/eXE0wZMUIJ',
    'expanded_url': 'https://twitter.com/NBA/status/1028396891682164736/video/1',
    'type': 

In [6]:
try:     
    # Creation of query method using appropriate parameters
    tweets =    tweepy.Cursor(api.user_timeline,id=username).items(count)
 
    
    # Pulling information from tweets iterable object and adding relevant tweet information in our data frame
    for tweet in tweets:
        df_user_tweets = df_user_tweets.append(
                          {'Created at' : tweet._json['created_at'],
                                       'User ID': tweet._json['id'],
                              'User Name': tweet.user._json['name'],
                                        'Text': tweet._json['text'],
                     'Description': tweet.user._json['description'],
                        'Likes': tweet.user._json['favorite_count'],
                      'Retweets': tweet.user._json['retweet_count'],
                           'Location': tweet.user._json['location'],
             'Followers Count': tweet.user._json['followers_count'],
                 'Friends Count': tweet.user._json['friends_count'],
               'Statuses Count': tweet.user._json['statuses_count'],
         'Profile Image Url': tweet.user._json['profile_image_url'],
                         }, ignore_index=True)
except BaseException as e:
    print('failed on_status,',str(e))
    time.sleep(3)

Unexpected parameter: id


failed on_status, 'favorite_count'


In [8]:
# this is V2 of twitter's API
# this counts towards the 2 million tweets per month cap
client = tweepy.Client(bearer_token=os.environ.get('bearer_token'))

In [141]:
start_time = datetime.now() - timedelta(days = 1)
df2 = client.search_recent_tweets(
    'nba',
    start_time = start_time,
    tweet_fields = ["id", "text", "created_at", "lang", "public_metrics"],
    expansions=['author_id'],
    user_fields = ["name","username","profile_image_url","url"],
    max_results = 100)

In [144]:
users = {u["id"]: u for u in df2.includes['users']}
df = pd.DataFrame()
for tweet in df2.data:
    if users[tweet.author_id]:
        user = users[tweet['author_id']]
        df = df.append(
            {
                "created_at": tweet.data["created_at"],
                "tweet_id": tweet.data["id"],
                "username": user["username"],
                "user_id": user["id"],
                # "user_id": tweet.data["user"]["id"],
                "tweet": tweet.data["text"],
                "likes": tweet.data['public_metrics']["like_count"],
                "retweets": tweet.data['public_metrics']["retweet_count"],
                'replies': tweet.data['public_metrics']['reply_count'],
                "language": tweet.data["lang"],
                "scrape_ts": datetime.now(),
                "profile_img": user['profile_image_url'],
                "url": f"https://twitter.com/twitter/statuses/{tweet.data['id']}",
            },
            ignore_index=True,
        )

df

,created_at,tweet_id,username,user_id,tweet,likes,retweets,replies,language,scrape_ts,profile_img,url
0,2022-07-13T01:38:04.000Z,1547032583208321024,Righteousp21,1.448133e+18,RT @TTBNoah: NBA YoungBoy trial active rn. He ...,0.0,131.0,0.0,en,2022-07-12 20:38:23.520995,https://pbs.twimg.com/profile_images/144813539...,https://twitter.com/twitter/statuses/154703258...
1,2022-07-13T01:38:04.000Z,1547032580725391360,woodleysean,3.461490e+08,@LucasJHann totally. these dudes are maniacs i...,1.0,0.0,0.0,en,2022-07-12 20:38:23.523957,https://pbs.twimg.com/profile_images/154362565...,https://twitter.com/twitter/statuses/154703258...
2,2022-07-13T01:38:04.000Z,1547032580465336320,StevieTweets20,1.031473e+09,RT @ShamsCharania: The NBA Board of Governors ...,0.0,476.0,0.0,en,2022-07-12 20:38:23.526566,https://pbs.twimg.com/profile_images/134775874...,https://twitter.com/twitter/statuses/154703258...
3,2022-07-13T01:38:03.000Z,1547032579127271429,_zay9x,3.741050e+08,RT @BleacherReport: The NBA Board of Governors...,0.0,198.0,0.0,en,2022-07-12 20:38:23.528631,https://pbs.twimg.com/profile_images/152988357...,https://twitter.com/twitter/statuses/154703257...
4,2022-07-13T01:38:03.000Z,1547032577126563840,_Trav0,2.299805e+09,@NBA_Skits Too small,0.0,0.0,0.0,en,2022-07-12 20:38:23.530767,https://pbs.twimg.com/profile_images/119154450...,https://twitter.com/twitter/statuses/154703257...
...,...,...,...,...,...,...,...,...,...,...,...,...
95,2022-07-13T01:37:30.000Z,1547032437925957633,WestbrookTop5PG,1.075459e+09,"Next time you see something outlandish, someth...",0.0,0.0,0.0,en,2022-07-12 20:38:23.735176,https://pbs.twimg.com/profile_images/147028317...,https://twitter.com/twitter/statuses/154703243...
96,2022-07-13T01:37:29.000Z,1547032435388555268,Righteousp21,1.448133e+18,RT @ybdeyo: NEW NBA YOUNGBOY MEDIAAAA!!! https...,0.0,86.0,0.0,en,2022-07-12 20:38:23.737246,https://pbs.twimg.com/profile_images/144813539...,https://twitter.com/twitter/statuses/154703243...
97,2022-07-13T01:37:29.000Z,1547032434365153281,_DesPain_,2.689326e+08,No need to shorten the season but let’s add mo...,0.0,0.0,0.0,en,2022-07-12 20:38:23.739327,https://pbs.twimg.com/profile_images/151134645...,https://twitter.com/twitter/statuses/154703243...
98,2022-07-13T01:37:29.000Z,1547032433979187200,bubblybrielle,1.212883e+08,Stay tuned 🍀 https://t.co/4dodvgQkJ2,0.0,0.0,0.0,en,2022-07-12 20:38:23.741420,https://pbs.twimg.com/profile_images/147635120...,https://twitter.com/twitter/statuses/154703243...


In [100]:
counts = client.get_recent_tweets_count(query='nba', granularity='day')

for count in counts.data:
    print(count)

{'end': '2022-07-07T00:00:00.000Z', 'start': '2022-07-06T01:23:02.000Z', 'tweet_count': 176186}
{'end': '2022-07-08T00:00:00.000Z', 'start': '2022-07-07T00:00:00.000Z', 'tweet_count': 190295}
{'end': '2022-07-09T00:00:00.000Z', 'start': '2022-07-08T00:00:00.000Z', 'tweet_count': 109645}
{'end': '2022-07-10T00:00:00.000Z', 'start': '2022-07-09T00:00:00.000Z', 'tweet_count': 100204}
{'end': '2022-07-11T00:00:00.000Z', 'start': '2022-07-10T00:00:00.000Z', 'tweet_count': 103093}
{'end': '2022-07-12T00:00:00.000Z', 'start': '2022-07-11T00:00:00.000Z', 'tweet_count': 104932}
{'end': '2022-07-13T00:00:00.000Z', 'start': '2022-07-12T00:00:00.000Z', 'tweet_count': 113927}
{'end': '2022-07-13T01:23:02.000Z', 'start': '2022-07-13T00:00:00.000Z', 'tweet_count': 9559}
